In [37]:
import torch

In [36]:
import numpy as np
from google.colab import drive

drive.mount('/content/drive')
fashion_train= '/content/drive/MyDrive/SIS420-INTELIGENCIA_ARTIFICIAL/DATASETs/fashion-mnist_train.csv'
data_train = np.loadtxt(fashion_train, skiprows=1, delimiter=',')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [38]:
X, Y = data_train[:,1:], data_train[:,0]

In [39]:
fashin_test = '/content/drive/MyDrive/SIS420-INTELIGENCIA_ARTIFICIAL/DATASETs/fashion-mnist_test.csv'
data_test = np.loadtxt(fashin_test, skiprows=1, delimiter=',')
x_test, y_test =  data_test[:,1:], data_test[:,0]

In [40]:
# normalización y split
X_train = X[:] / 255
X_test = x_test[:] / 255
y_train = Y[:].astype(np.int)
y_test = y_test[:].astype(np.int)
# X_train, X_test, y_train, y_test = X[:50000] / 255., x_test[50000:] / 255., Y[:50000].astype(np.int), y_test[50000:].astype(np.int)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note lin

In [41]:
D_in, H, D_out = 784, 100, 10

# convertimos datos a tensores y copiamos en gpu
X_t = torch.from_numpy(X_train).float().cuda()
Y_t = torch.from_numpy(y_train).long().cuda()

model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)#.to("cuda")


In [42]:
outputs = model(torch.randn(64, 784))
outputs.shape

torch.Size([64, 10])

In [43]:
model.to("cuda")

Sequential(
  (0): Linear(in_features=784, out_features=100, bias=True)
  (1): ReLU()
  (2): Linear(in_features=100, out_features=10, bias=True)
)

In [44]:
# función de pérdida y derivada

def softmax(x):
    return torch.exp(x) / torch.exp(x).sum(axis=-1,keepdims=True)

# def cross_entropy(output, target):
#     logits = output[torch.arange(len(output)), target]
#     loss = - logits + torch.log(torch.sum(torch.exp(output), axis=-1))
#     loss = loss.mean()
#     return loss

In [45]:
from sklearn.metrics import accuracy_score

def evaluate(x):
    model.eval()
    y_pred = model(x)
    y_probas = softmax(y_pred)
    return torch.argmax(y_probas, axis=1)

y_pred = evaluate(torch.from_numpy(X_test).float().cuda())
accuracy_score(y_test, y_pred.cpu().numpy())

0.0645

In [47]:

criterion = torch.nn.CrossEntropyLoss() # funcion de perdida
optimizer = torch.optim.SGD(model.parameters(), lr=0.8) # optimizador desenso por el gradiente

epochs = 100
log_each = 10
l = []
model.train()
for e in range(1, epochs+1): 
    
    # forward
    y_pred = model(X_t)

    # loss
    loss = criterion(y_pred, Y_t)
    l.append(loss.item())
    
    # ponemos a cero los gradientes
    optimizer.zero_grad()

    # Backprop (calculamos todos los gradientes automáticamente)
    loss.backward()

    # update de los pesos
    optimizer.step()
    
    if not e % log_each:
        print(f"Epoch {e}/{epochs} Loss {np.mean(l):.5f}")
        
y_pred = evaluate(torch.from_numpy(X_test).float().cuda())
accuracy_score(y_test, y_pred.cpu().numpy())

Epoch 10/100 Loss 0.40754
Epoch 20/100 Loss 0.40469
Epoch 30/100 Loss 0.40917
Epoch 40/100 Loss 0.40793
Epoch 50/100 Loss 0.40803
Epoch 60/100 Loss 0.40768
Epoch 70/100 Loss 0.40605
Epoch 80/100 Loss 0.40605
Epoch 90/100 Loss 0.40606
Epoch 100/100 Loss 0.40576


0.8553